# Problem 1
Recall the procedure of ECDSA

1. $p, q, E, P \leftarrow \mathop{\text{PGen}}(1^\lambda)$, where $p$, $q$ are prime numbers, $E = E(\mathbb{F}_p)$ is an elliptic curve over the finite field $\mathbb{F}_p$, and $P$ is a base point within the group of points with prime order $q$
2. The secret key $\alpha$ is uniformly sampled from the multiplicative group $\mathbb{Z}_q^\times$, and the public key is $\alpha P$.
3. To encrypt a message:
    - sample a random $k \leftarrow \mathbb{Z}_q^\times$. 
    - compute $kP$, and set $r$ to be the x coordinate of $kP$ reduced by $q$: $r \leftarrow x(kP) \mod q$
    - compute $s \leftarrow k^{-1}(H(m) + \alpha r) \mod q$
    - the signature is $\sigma = (r, s)$

## a)
If Alice the same $k$ for two distinct signatures on two distinct messages, then the first half of the signature will be identical. An eavesdropper can thus discover that the same $k$ has been used for two distinct signatures

## b)
Suppose $\sigma_1 = (r_1, s_1), \sigma_2 = (r_2, s_2)$ are two signatures on two distinct messages signed using the same secret key $\alpha$ and the same ephemeral secret $k$, then from part (a) we know that $r_1 = r_2$, and from the signing algorithm of ECDSA, we know that

$$
\begin{aligned}
s_1 &= k^{-1}(H(m_1) + \alpha r_1) \mod q \\
s_2 &= k^{-1}(H(m_2) + \alpha r_2) \mod q \\
\end{aligned}
$$

Notice that $s_1, s_2, r_1, r_2, H(m_1), H(m_2)$ are all public knowledge, so we have a two equations with two unknowns $k, \alpha$, from which we can solve both of them.

First, if we subtract the first equation from the second, we get

$$
\begin{aligned}
s_1 - s_2 &\equiv k^{-1}(H(m_1) + \alpha r_1) - k^{-1}(H(m_2) + \alpha r_2) \mod q\\ 
&\equiv k^{-1}(H(m_1) - H(m_2)) \mod q
\end{aligned}
$$

From this we can compute $k \equiv (H(m_1) - H(m_2)) \cdot (s_1 - s_2)^{-1} \mod q$. With $k$ recovered, we can use $k$ to recover the secret key:

$$
\alpha \equiv (s_1k - H(m_1)) \cdot r_1^{-1} \mod q
$$

<p style="page-break-after:always;"></p>

# Problem 2

## a)
With polynomial long division we can compute the reduction of $x^6 + 4 x^4 + 3x^3 + x + 1$ by $x^4 + 1$:

$$
x^6 + 4x^4 + 3x^3 + x + 1 = (x^2 + 4)(x^4 + 1) + (3x^3 - x^2 + x - 3)
$$

Therefore we have

$$
x^6 + 4x^4 + 3x^3 + x + 1 \equiv 3x^3 - x^2 + x - 3 \mod (x^4 + 1)
$$

## b)
Observe the following two computation (within the polynomial ring $R = \mathbb{Z}_q[x]/\langle f(x) \rangle$):

$$
\begin{aligned}
c &= s^\prime b + e^{\prime\prime} + \lfloor \frac{q}{2} \rfloor m \\
&= s^\prime (as + e) + e^{\prime\prime} + \lfloor \frac{q}{2} \rfloor m \\
&= (s^\prime a)s + s^\prime e + e^{\prime\prime} + \lfloor \frac{q}{2} \rfloor m \\
b^\prime s &= (s^\prime a + e^\prime)s \\
&= (s^\prime a)s + e^\prime s
\end{aligned}
$$

Subtract the second equation from the first, we have:

$$
c - b^\prime s = (s^\prime e - e^\prime s) + e^{\prime\prime} + \lfloor \frac{q}{2} \rfloor m
$$

Knowing that $s, e, s^\prime, e^\prime$ are all sampled from the same probability distribution, the expectation of $(s^\prime e - e^\prime s)$ is 0. On the other hand, if $\chi$ produces small coefficients with high probability, then $e^{\prime\prime}$ will be a relatively small term with respect to $\lfloor \frac{q}{2} \rfloor$ such that the result of rounding $\lfloor \frac{q}{2} \rfloor$ is unaffected with or without the error term. This means that with high probability:

$$
c - b^\prime s \approx \lfloor \frac{q}{2} \rfloor m
$$

from where we can round the coefficients of $c - b^\prime s$: for each coefficient, if the value is closer to $q$, then the original coefficient is $1$, otherwise the original coefficient is $0$.

To summarize, the decryption algorithm is as follows:

1. Compute $d = c - b^\prime s$ as an element of the quotient ring $\mathbb{Z}_q[x]/\langle x^n + 1 \rangle$
2. Round the coefficients of $d$: for each coefficient, round it to $1$ if the value is closer to $q$, otherwise round it to 0.
3. $m = \lfloor d \rceil$ is the decryption

## c)
Here is the code I used to compute the decryption

```python
from sympy import symbols
from sympy.polys.domains import GF

x = symbols("x")
Zq = GF(113)
Rq = Zq[x]
modulus = Rq(x ** 3 + 1)

c = Rq(105 * x ** 2 + 64 * x + 104)
b_prime = Rq(62 * x ** 2 + 50 * x + 64)
s = Rq(1 + x - x ** 2)
m = (c - b_prime * s).rem(modulus)
print(m)  # 57 mod 113*x**2 + x + 52 mod 113 -> [57, 1, 52] -> [1, 0, 1]
```

The result is $c - b^\prime s =  57x^3 + x + 52$. After the rounding is applied, we get $m[x] = x^3 + 0 + 1$, so the original message is `0b101`

## d)
As was described in part (b), the rounding of $c - b^\prime s$ produces a correct decryption if and only if the error terms are not too large. In the context of this question, we have $q = 113$ and the error term not exceeding 1 in absolute value. As a result, adding a few error terms to $m$ will not affect how it rounds. Therefore, the decryption is guaranteed to be correct.

<p style="page-break-after:always;"></p>

# Problem 3

## a)
There are two main things to check. First we need to check the authenticity of the message against the signer's public key, then we need to check the authenticity of the signer's certificate. Note that there are many ways the certificate can be invalid, including being expired, being signed by someone other than the certificate authority, or not passing the verification using the certificate authority's public key.

To summarize, the algorithm goes as follows:

1. Verify the signature of the message against the claimed signer's public key. If the verification fails, the message cannot be trusted
2. Check that the claimed signer's certificate is not expired. If the certificate expires, the message cannot be trusted
3. Check that the claimed signer's certificate is issued by the certificate authority. If the certificate is not issued by the certificate authority, the message cannot be trusted
4. Verify the certificate's signature against the certificate authority's public key. If the verification fails, the message cannot be trusted

## b)
```python
from datetime import datetime
import json
import sys
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.utils import encode_dss_signature
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature

# Helper function for converting strings into byte arrays needed by cryptographic functions
def string_to_bytes(s):
    return s.encode('utf-8')

# This function will ensure that we represent the JSON dictionary as exactly the
# same string every time, otherwise we'd get different hashes while signing
def canonicalize_json(j):
    return json.dumps(j, sort_keys=True)

def verify(ca_identity, signed_message_filename): 

    print("Trying to verify " + signed_message_filename)

    # Load the signed message data
    with open(signed_message_filename, 'r') as fh:
        signed_message = json.load(fh)

    # Read out the identity of the signer and load their certificate
    signer_identity = signed_message['signer identity']
    with open(signer_identity + '.cert', 'r') as fh:
        signer_cert = json.load(fh)
    # Format the certificate body for signing as a byte array in a canonical order
    cert_body_to_be_signed = string_to_bytes(canonicalize_json(signer_cert["body"]))

    # Read out the identity of the issuer and load their public key
    issuer_identity = signer_cert['body']['issuer identity']
    signer_pk = serialization.load_pem_public_key(string_to_bytes(signer_cert['body']['public key']))
    with open(ca_identity + '.pk', 'r') as fh:
        ca_public_key = serialization.load_pem_public_key(string_to_bytes(fh.read()))

    # YOUR SOLUTION STARTS HERE
    msg_body = signed_message["message"]
    msg_sig_raw = signed_message["signature"]
    msg_sig = encode_dss_signature(msg_sig_raw["r"], msg_sig_raw["s"])
    ecdsa = ec.ECDSA(hashes.SHA256())
    try:
        # Check that the message is authentic
        signer_pk.verify(msg_sig, string_to_bytes(msg_body), ecdsa)
    except InvalidSignature:
        print(f"The message is not authentic against {signer_identity}'s public key")
        return False

    validity_start = datetime.fromisoformat(signer_cert["body"]["validity start"])
    validity_end = datetime.fromisoformat(signer_cert["body"]["validity end"])
    if datetime.now() < validity_start or datetime.now() > validity_end:
        print("Certificate has expired")
        return False
    if issuer_identity != "dstebila":
        print("Certificate is not signed by dstebila")
        return False
    cert_sig = encode_dss_signature(
        signer_cert["signature"]["r"],
        signer_cert["signature"]["s"],
    )
    try:
        ca_public_key.verify(cert_sig, cert_body_to_be_signed, ecdsa)
    except InvalidSignature:
        print(f"Certificate is not authentic")
        return False

    print("Message can be trusted")

verify("dstebila", "message1.signed.txt")
verify("dstebila", "message2.signed.txt")
verify("dstebila", "message3.signed.txt")
verify("dstebila", "message4.signed.txt")
verify("dstebila", "message5.signed.txt")

```

## c)
The output of the program above is as follows:

```
Trying to verify message1.signed.txt
Message can be trusted
Trying to verify message2.signed.txt
The message is not authentic against jeanne's public key
Trying to verify message3.signed.txt
Certificate is not authentic
Trying to verify message4.signed.txt
Certificate has expired
Trying to verify message5.signed.txt
Certificate is not signed by dstebila
```

1. Message 1 can be trusted
2. Message 2 cannot be trusted because its signature cannot be verified against the signer's public key
3. Message 3 cannot be trusted because the certificate's signature cannot be verified against `dstebila`'s public key
4. Message 4 cannot be trusted because the certificate has expired
5. Message 5 cannot be trusted because the certificate is not signed by `dstebila`

<p style="page-break-after:always;"></p>

# Problem 4
Without the offline QR code scanning, the two parties are blindly trusting that the public keys of the other party, supplied by the server, are authentic. This means that an attacker can impersonate the server and replace authentic public keys with the attacker's public key, then relays the initial message exchanged. From here, the attacker can intercept and manipulate Alice's and Bob's message, and Alice and Bob could not have detected that they are not actually talking to each other.

<p style="page-break-after:always;"></p>

# Problem 5

## a)
Because HMAC-SHA256 is a secure PRF, its output is computationally indistinguishable from truly random sequences, so if the increments are implemented correctly, then the newest output of the HOTP should be independent from all previously generated output. This means that an attacker will have no better way than to blindly guess at the 6-digit HOTP (the smallest key size ofr HMAC-SHA256 is 256 bits, so brute-forcing the secret key is not computationally feasible).

Assuming that the server rate-limit guesses to 10 guesses, the chance of guessing the correct HOTP is $10^{-5}$

## b)
If the HOTP counter is fixed at $0$, then after the attacker obtains one HOTP, he/she can simply reuse the HOTP in subsequent attacks, thus rendering the HOTP completely useless. In production, this system will definitely be vulnerable as users are expected to frequently sign in and sign off, which requires many uses of the HOTP, thus creating many opportunities for the attacker to intercept the HOTP.

## c)
With TOTP, the counter is replaced with "the number of time intervals (typically 30 seconds) passed since a pre-determined starting time (typically UNIX time 0)". Since TOTP is based on HOTP, they should be equally secure. However, TOTP is easier to use than HOTP because with TOTP, the client and the server do not need to explicitly synchronize their counter at every login. Instead, they only need to synchronize their settings once at registration, and afterwards each can correctly compute the time-based counter independently.


<p style="page-break-after:always;"></p>

# Problem 6

## a)

### 1)
For a given challenge, assuming that each of the $2^{40}$ possible keys produces a uniformly response from the $2^{24}$ possible responses, then on average there are $2^{16}$ keys that map the given challenge to any specific response.

### 2)
Suppose that for a given challenge-response pair, a key is found to map the challenge to the response. By the argument above, because there are on average $2^{16}$ keys that map any specific challenge to any specific response, the probability that the recovered key is the correct key is $2^{-16}$. In other words, if we have a key that maps a known challenge to a response, then the probability that this key is NOT the correct key is $1 - 2^{-16}$.

Assuming that each challenge-response pair is evaluated independently, the chance that a key satisfies $n$ pairs and is still not the correct key is thus $(1 - 2^{-16})^n$. By trial and errors, I found that for $n = 2^{15}$, this probability is 0.6065, but for $n = 2^{16}$, this probability is 0.3679, meaning that if we found a key that satisfies $2^{16}$ challenge-response pair, then it is more likely than not that such a key is the true secret key.

### 3)
For the key recovery attack, I need $2^{16}$ challenge-response pairs where each challenge is distinct. The challenge of the first challenge-response pair is the public challenge in the lookup table $\mathop{\text{ch}}_\text{pub}$, and the remaining challenges simply increment on the previous challenge:

$$
\text{chosen pairs} = \{(\text{ch}_\text{pub} + i, \text{DST40}(\text{sk}, \text{ch}_\text{pub} + i))  \mid \; 0 \leq i < 2^{16}\}
$$

Using the response of the first pair, look up the set of possible keys that map from $\text{ch}_\text{pub}$ to $\text{resp}_0$, then for each of the key, check if the key is a valid key for the remaining chosen challenge-response pairs.

Assuming that it takes 2 seconds to perform $2^{16}$ DST40 operations, it takes 2 seconds to check any specific key against all $2^{16}$ chosen challenge-response pairs. Since on average there are $2^{16}$ keys that map any specific challenge to any specific response, we will need to check $2^{16}$ keys, which in total takes $2^{17}$ seconds (or approximately 36 hours).

## b)
First we use a radio receiver near the car to capture the car ID, which is feasible because the car repeatedly broadcasts this ID.

Then, when we are close to the key fob, we broadcasts the captured car ID and the chosen challenges described in part (a.3), then we receive and store the response from the key fob. Thus we have obtained the chosen challenge-response pair described in part (a.3).

From here we run the key recovery attack offline and recover the secret key and can thus clone the key fob.

